In [9]:
#Подключаем гугл диск
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Импортируем все необходимые библиотеки
import os
import cv2
from PIL import Image, ImageDraw
import math
import shutil


In [0]:
#Пункт 1. Обработка типовок
#получаем список все изображений в папке
def get_files(save_w, way, tipovki_list):
    os.chdir(way)
    folders = os.listdir() #Получаем содержимое папки
    for i in range (0, len(folders)):
        if str(folders[i]).count('_') != True: #Если в названии папки нет знака "_"
            os.chdir(way + str(folders[i]) + '/') #переходим в эту папку
            tipovki_list.append(folders[i]) #в список tipovki_list добавляем название папки
            tipovki_list.append(os.listdir()) #в список tipovki_list добавляем список с именами файлов в папке
            #В папке, для обработанных изображений создадим (если их там нет) папки с названием ЖК и знаком "_" в конце
            os.chdir(save_w)
            try:
                os.mkdir(folders[i] + '_')
            except:
                pass
    return tipovki_list

#Обрабатываем изображение
def ghosts(direct, direct_save, name):
    os.chdir(direct)
    im = Image.open(name) #загружаем изображение
    im = im.convert("RGBA") #конвертируем изображение в RGB с альфа-каналом 
    width = im.size[0] #Определяем ширину изображения 
    height = im.size[1] #Определяем высоту изображения
    pix = im.load() #Выгружаем значения пикселей изображения

    #делаем границы прозрачными
    opacity_level = 0 # Задаем уровень прозрачности границ (вводится значения от 0 (абсолютно прозрачный) до 255 (абсолютно не прозрачный))
    center = (0, 2)
    yellow = (255, 255, 0, 0)
    ImageDraw.floodfill(im, center, yellow, thresh = 10) #заливаем изображение  желтым цветом с альфа каналом == 0, начиная со 2 пикселя 
    
    
    #обрезаем изображение
    left, up = 0, 0
    min_j, max_j = 0, height
    right, down = 0, 0
    min_i = 0 
    for i in range (width):
        for j in range (height):
            #print (pix[i, j][3])
            if pix[i, j][3] > 100 and left == 0: #ищем левую границу
                left = i 
                max_j =  j
            elif pix[i, j][3] > 100 and left != 0: 
                if j < max_j: #ищем верхнюю границу
                    up = j
                    max_j = j
                elif j > min_j: #ищем нижнюю границу
                    down =  j
                    min_j = j 
                elif i > min_i: #ищем правую границу
                    right = i
                    min_i = i
    area = (left, up, right, down)
    im = im.crop(area)
    
    #меняем размер изображения до 170*170 pix
    im2 = Image.new('RGBA', (170, 170), color=(255, 255, 255, 0))
    width2, height2 = im2.size[0], im2.size[1]
    canvas = 170 #Размер нового полотна (изображение, все пиксели белого цвета) в пикселях
    width_changed = im.size[0] #Определяем ширину изображения 
    height_changed = im.size[1] #Определяем высоту изображения
    #Изменяем размер исходного изображения зависимости от размеров сторон исходного изображения
    #(изменённое изображение вставляем в новое полотно)
    if width_changed > canvas and height_changed > canvas:
        im = im.resize((canvas, canvas), Image.BILINEAR)
        im2.paste(im)
    elif width_changed > canvas and height_changed < canvas:
        im = im.resize((canvas, canvas), Image.BILINEAR)
        im2.paste(im)
    elif width_changed > canvas and height_changed == canvas:
        im = im.resize((canvas, height_changed), Image.BILINEAR)
        im2.paste(im, box=(0, height2 - height_changed))
    elif width_changed < canvas and height_changed > canvas:
        im = im.resize((canvas, canvas), Image.BILINEAR)
        im2.paste(im)
    elif width_changed < canvas and height_changed < canvas:
        im = im.resize((canvas, canvas), Image.BILINEAR)
        im2.paste(im)
    elif width_changed < canvas and height_changed == canvas:
        im = im.resize((canvas, height_changed), Image.BILINEAR)
        im2.paste(im, box=(0, height2 - height_changed))
    elif width_changed == canvas and height_changed > canvas:
        im = im.resize((width_changed, canvas), Image.BILINEAR)
        im2.paste(im, box=(width2 - width_changed, 0))
    elif width_changed == canvas and height_changed < canvas:
        im = im.resize((width_changed, canvas), Image.BILINEAR)
        im2.paste(im, box=(width2 - width_changed, 0))
    elif width_changed == canvas and height_changed == canvas:
        im2.paste(im, box=(width2 - width_changed, height2 - height_changed))

    #Сохраняем изображение в формате .PNG 
    os.chdir(direct_save)
    im2.save(name[ : -4] + '_edit.png', "PNG")

#Объявляем функцию обработки типовок
def tipovki():
    img_list, tipovki_list = [], []
    way = '/content/drive/My Drive/Colab Notebooks/examples/tipovki_to_update/' #Здесь нужно указать путь к папкам, содержащим папки с исходными изображениями типовок
    save_w = '/content/drive/My Drive/Colab Notebooks/examples/tipovki/' #путь к папке, в которую нужно сохранить папки, содержащие обработанные изображения типовок
    get_files(save_w, way, tipovki_list) #выполняем функцию получения списка файлов
    for i in range (0, len(tipovki_list), 2):
        for j in range (0, len(tipovki_list[i + 1])):
            name = tipovki_list[i + 1][j] #name -строка состоящая из названия файла
            direct = way + tipovki_list[i] + '/' #путь к папке с исходными изображениями
            direct_save = save_w + tipovki_list[i] + '_' + '/' #путь к папке для сохранения обработанных изображений
            ghosts(direct, direct_save, name)

#Обработка планировок   
way = '/content/drive/My Drive/Colab Notebooks/examples/planirovki_to_update/' #Здесь нужно указать путь к папкам, содержащим папки с исходными изображениями планировок
save_w = '/content/drive/My Drive/Colab Notebooks/examples/planirovki/'#путь к папке, в которую нужно сохранить папки, содержащие обработанные изображения планировок 


#Пункт 2.1
arr, arr_c = (0, 0, 0), (0, 0, 0)
img_list = []
k, no_kompas = 1, 0


def get_files_two_point_one(planirovki_list, way): #Функция будет возвращать список всех папок в каталоге и файлов в этих папках
    os.chdir(way)
    fold = os.listdir()
    folders = []
    for d in fold:
        if os.path.isdir(d) == True:
            folders.append(d)
    for i in range (0, len(folders)):
        if str(folders[i]).count('_2.1') != True and str(folders[i]) != 'kompas_white.png':
            os.chdir(way + str(folders[i]) + '/')
            
            shutil.copy('/content/drive/My Drive/Colab Notebooks/examples/planirovki_to_update/kompas_white.png', way + str(folders[i]) + '/') #Копируем файл kompas_white  в каждую папку с планировками
            
            planirovki_list.append(folders[i])
            planirovki_list.append(os.listdir()) 
    return planirovki_list

def temp_kompas(name, arr, direct): #функция создаёт изображение-шаблон (с компасом) из основного изображения
    global k #счетчик кол-ва компасов для планировок в текущей папке
    global no_kompas #триггер наличия компаса
    global arr_c #координаты компаса
    os.chdir(direct) #переходим в папку с планировками
    im = Image.open(name) #открываем изображение 
    im2 = im.transpose(Image.FLIP_TOP_BOTTOM).convert('RGB') #переворачиваем изображение и конвертируем его в RGB формат
    width = im2.size[0] #Определяем ширину изображения
    height = im2.size[1] #Определяем высоту изображения
    y, end = 0, 0 #y - расстояние от края до верхней границы перевернутого изображения; end - счетчик 
    pix = im2.load() #Выгружаем значения пикселей
    pix_im = im.load() #Выгружаем значения пикселей
    center = (0, 2)
    white = (255, 255, 255, 0) #белый с альфа каналом == 0 (белый, абсолютно прозрачный)
    
    #ищем границу первый не белый пиксель в середине изображения (начинаем с середины - чтобы не наткнуться на компас)
    i = width // 2
    for j in range (0, height):
        if pix[i, j] != (255, 255, 255) and end == 0:
            y = j #нижняя граница помещения
            end = 1
            break 
    if y >= 80: #если от нижней границы полотна до первой стены больше 80 пикселей
        #обрезаем изображение (чтобы получить только компас)
        im = im.crop((0, height - 90, 95, height)).convert('RGBA') #судя по изображениям максимальная высота 
        #от края изображения до высшей точки компаса - 90 pix, максимальная ширина от левой границы 
        #до последней точки компаса - 95 pix     
        ImageDraw.floodfill(im, center, white, thresh = 10) #заливаем поля компаса прозрачным (белым, с альфа каналом)
        kompas = im
        width = kompas.size[0]  
        height = kompas.size[1] 
        left, up = 0, 0
        min_j, max_j = 0, height
        right, down = 0, 0
        min_i = 0 
        pix = kompas.load()
        for i in range (width):
            for j in range (height):
                #идем по рядам изображения (массив координат и цветов) сверху вниз. Если пиксель не прозрачный - пиксель граничный
                if pix[i, j][3] > 100 and left == 0:#ищем левую границу: 
                    left = i 
                    max_j =  j
                elif pix[i, j][3] > 100 and left != 0: 
                    if j < max_j: #ищем верхнюю границу
                        up = j
                        max_j = j
                    elif j > min_j: #ищем нижнюю границу
                        down =  j
                        min_j = j 
                    elif i > min_i: #ищем правую границу
                        right = i
                        min_i = i
        arr = (left - 3, up - 3, right + 3, down + 3) #кортеж содержащий координаты начальной и конечной граничных точек + 3 пикселя в ширину и в высоту.
        #Нужен для обрезания компаса в шаблон
        arr_c = arr 
        
        if pix[10, height - 10][3] < 100: #если фон у изображения изначально прозрачный - фон компаса заливаем белым прозрачным 
            white = (255, 255, 255, 0)
        else:
            white = (255, 255, 255) #иначе - фон компаса заливаем белым
        
        #Заливаем компас с 4-х сторон
        width = kompas.size[0] #Определяем ширину обрезанного изображения (компаса)
        height = kompas.size[1] #Определяем высоту обрезанного изображения (компаса)
        left_down = (2, height - 2) #координаты нижнего левого угла
        right_down = (width - 2, height - 2) #координаты нижнего правого угла
        right_up = (width - 2, 2) #координаты верхнего правого угла
        ImageDraw.floodfill(kompas, center, white, thresh = 10) #заливаем белым верхнюю левую часть компаса
        ImageDraw.floodfill(kompas, left_down, white, thresh = 10) #заливаем белым нижнюю левую часть компаса
        ImageDraw.floodfill(kompas, right_down, white, thresh = 10) #заливаем белым верхнюю правую часть компаса
        ImageDraw.floodfill(kompas, right_up, white, thresh = 10) #заливаем белым нижнюю правую часть компаса
        kompas1 = kompas.crop(arr_c).convert('RGB') #обрезаем изображение и конвертируем его в RGB формат (убираем альфа-канал)
        
        #Обрезаем и сохраняем изображения-шаблоны (содержащие компас)
        os.chdir(direct) #переходим в каталог с планировками
        st = set(kompas1.getdata()) #получаем значения цветов каждого пикселя и сохраняем их как множество (т.к. в множестве одинаковые элементы не дублируются)
        if len(st) > 3: #если в изображении kompas1 больше 3 цветов (множество содержит более 3 кортежей т.е. это не белый фон) 
            no_kompas = 0 #триггер наличия компаса в изображении
            kompas1.save('kompas_' + str(k) + '.png', 'PNG') #сохраняем новый компас как шаблон  
            k += 1
            arr = (left - 5, up - 5, right + 5, down + 5) #к граничным точкам добавляем + 5 пикселей в ширину и в высоту
            arr_c = arr  
            kompas2 = kompas.crop(arr_c).convert('RGB') #обрезаем компас и убираем альфа-канал
            kompas2.save('kompas_' + str(k) + '.png', 'PNG') #сохраняем новый компас как шаблон
            k +=1
            arr = (left - 1, up - 1, right + 1, down + 1) #к граничным точкам добавляем + 1 пиксель в ширину и в высоту
            arr_c = arr  
            kompas3 = kompas.crop(arr_c).convert('RGB') #обрезаем компас и убираем альфа-канал
            kompas3.save('kompas_' + str(k) + '.png', 'PNG') #сохраняем новый компас как шаблон
            k += 1
            arr = (left, up, right, down) #кортеж с гарничными точками (компас будет обрезан строго по ним)
            arr_c = arr  
            kompas = kompas.crop(arr_c).convert('RGB') #обрезаем компас и убираем альфа-канал
            kompas.save('kompas_' + str(k) + '.png', 'PNG') #сохраняем новый компас как шаблон
            k += 1            
    else: #если расстояние от верхней стены (т.к. изображение перевернуто) до верхнего края изображения меньше 80 пикселей - компаса нет 
        no_kompas = 1           
    return arr_c   
    
def find_kompas(name, direct, direct_save): 
    global no_kompas
    global arr_c
    os.chdir(direct)
    kompas_list = []
    method = cv2.TM_SQDIFF_NORMED #метод сравнения шаблона и изображения
    high, low = [], []
    tree = os.listdir(direct) #получаем список файлов в каталоге
    for tr in tree: #проходим по списку 
        if tr.count('kompas') > 0: #если значение элемента списка содержит слово "kompas" 
            kompas_list.append(tr) #добавляем его в список kompas_list
    kompas_list = sorted(kompas_list) #сортируем список, чтобы файл kompas_white был последним в списке
    opacity_level = 0 # Задаем значение прозрачности слоя: вводится значение от 0 до 255, где 0 - абсолютно прозрачный, 255 - не прозрачный
    center = (0, 2)
    yellow = (255, 255, 0, 0)
    second = 0 #триггер, second == 0 означает, что поиск по шаблону для данного изображения не выполнялся
    
    #Читаем изображения
    large_image = cv2.imread(name) #открываем изображение в котором будем искать изображение-шаблон (компас) для модуля openCV
    im = Image.open(name) #открываем изображение в котором будем искать изображение-шаблон (компас) для модуля PIL
    im = im.convert('RGBA') #конвертируем изображение в кодировку с альфа каналом
    width = im.size[0] #Определяем ширину изображения 
    height = im.size[1] #Определяем высоту изображения 
    pix = im.load() #Выгружаем значения пикселей изображения
    
    #Закрашиваем компас (если он есть)
    os.chdir(direct) #переходим в папку с изображением
    for templ in kompas_list: #идем по списку компасов
        small_image = cv2.imread(templ) #открываем изображение шаблон для модуля openCV
        result = cv2.matchTemplate(large_image, small_image, method) #поиск части изображения в изображении, с заданым методом
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result) #min_val в данном случае наилучшее совпадение, max_val - максимальное значение, min_loc - местоположение 
        #минимального значения координаты из входного изображения), max_loc- местоположение максимального значения (координаты из входного изображения)
        #здесь min_val -координаты точки с наилучшим совпадением т.к. мы используем метод TM_SQDIFF_NORMED
        
        if min_val <= 0.01: #min_val значение наилучшего совпадения если значение меньше 0.01 - по другим шаблонам не ищем
            if templ != 'kompas_white.png': #если изображение-шаблон не kompas_white.png (белый прямоугольник)
                mn,_,mnLoc,_ = cv2.minMaxLoc(result) 
                MPx,MPy = mnLoc #получаем координаты верхнего левого угла вхождения изображения-шаблона в изображение
                trows,tcols = small_image.shape[:2]
                high.append(MPx) #первая координата начала компаса 
                high.append(MPy) #втораяя координата начала компаса
                low.append(MPx + tcols) #первая координата конца компаса
                low.append(MPy + trows) #вторая координата конца компаса 
                #меняем каждый пиксель компаса на белый, прозрачный
                for i in range (high[0], low[0]):
                    for j in range (high[1], low[1]):
                        im.putpixel((i, j), (255, 255, 255, 0))
                #Дополнительно зальем нижнюю часть изображения (до середины) белым, прозрачным т.к. могли остаться обозначения сторон света
                for w in range (width // 2):
                    for h in range (height - 40, height):
                        im.putpixel((w, h), (255, 255, 255, 0))
                break
            elif templ == 'kompas_white.png' and second == 0: #если наилучшее совпадение у шаблона с белым прямоугольником и это первый поиск вхождения для данного изображения
                im2 = im.transpose(Image.FLIP_TOP_BOTTOM).convert('RGB') #переворачиваем изображение и конвертируем его в rgb
                width2 = im2.size[0] #Определяем ширину изображения 
                height2 = im2.size[1] #Определяем высоту изображения
                end = 0 #триггер найдена ли граница помещения
                pix2 = im2.load() #Выгружаем значения пикселей изображения
                #ищем границу первый не белый пиксель в середине изображения (середина - чтобы не наткнуться на компас)
                ii = width2 // 2
                yy = 0
                for j in range (0, height2):
                    if pix2[ii, j] != (255, 255, 255) and end == 0: #если пиксель не белый
                        yy = j #j - нижняя граница помещения
                        end = 1
                        break
                
                if yy >= 80: #Если от края изображения до нижней границы помещения больше 80 пикселей 
                    temp_kompas(name, arr, direct) #применяем функцию создания изображения-шаблона (с компасом) из основного изображения
                    #обновляем список с названиями файлов-шаблонов (т.к. функция temp_kompas) возможно создала изображения-шаблоны
                    kompas_list = []
                    tree = os.listdir()
                    for tr in tree:
                        if tr.count('kompas') == True:
                            kompas_list.append(tr)
                    kompas_list = sorted(kompas_list)
                    #no_kompas - триггер наличия компаса в изображении
                    if no_kompas != 1: #если компас в изображении найден
                        #закрашиваем компас
                        for templ in kompas_list: #идем по списку компасов
                            small_image = cv2.imread(templ)
                            result = cv2.matchTemplate(large_image, small_image, method) 
                            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result) 
                            if min_val <= 0.01:
                                if templ != 'kompas_white.png':
                                    mn,_,mnLoc,_ = cv2.minMaxLoc(result)
                                    MPx,MPy = mnLoc
                                    trows,tcols = small_image.shape[:2]
                                    high.append(MPx) #первая координата начала компаса 
                                    high.append(MPy) #втораяя координата начала компаса
                                    low.append(MPx + tcols) #первая координата конца компаса
                                    low.append(MPy + trows) #вторая координата конца компаса 
                                    for i in range (high[0], low[0]):
                                        for j in range (high[1], low[1]):
                                            im.putpixel((i, j), (255, 255, 255, 0))                            
                                    for w in range (width // 2):
                                        for h in range (height - 40, height):
                                            im.putpixel((w, h), (255, 255, 255, 0))
                                    second = 1
                                    no_kompas = 0
                                    break             
                elif yy < 80: #Если от края изображения до нижней границы помещения меньше 80 пикселей - в изображении нет компаса 
                    no_kompas = 1
                    
    imc = im.convert('RGBA') #imc = изображение, конвертированное в RGBA формат
    pix_im = imc.load() #выгружаем значения пикселей изображения imc
    pix_im = pix_im[2, 2][3] #получаем значение альфа-канала пикселя с координатами (2, 2)
    #делаем границы прозрачными   
    if pix_im > 100: #если фон не прозрачный    
        ImageDraw.floodfill(im, center, yellow, thresh = 10) #заливаем все прозрачным фоном начина с 2 пикселя !Внимание на thresh 
        #(thresh - Необязательное пороговое значение, которое задает максимально допустимое отличие значения цвета пикселя от пикселя цветом, которого заливаем фон)!
        
    #обрезаем изображение
    im = im.convert('RGBA') #конвертируем изображение в формат RGBA (RGB с альфа-каналом)
    width = im.size[0] 
    height = im.size[1] 
    left, up = 0, 0
    min_j, max_j = 0, height
    right, down = 0, 0
    min_i = 0 
    #идем по рядам изображения (массив координат и цветов) сверху вниз. Если пиксель не прозрачный - пиксель граничный
    for i in range (width):
        for j in range (height):
            if pix[i, j][3] > 100 and left == 0: #ищем левую границу
                left = i 
                max_j =  j
            elif pix[i, j][3] > 100 and left != 0: 
                if j < max_j: #ищем верхнюю границу
                    up = j
                    max_j = j
                elif j > min_j: #ищем нижнюю границу
                    down =  j
                    min_j = j 
                elif i > min_i: #ищем правую границу
                    right = i
                    min_i = i
    area = (left, up, right, down)
    im = im.crop(area) #обрезаем изображение по граничным пикселям
    
    #Растянуть размер полотна не меняя размер изображения нельзя, поэтому, чтобы идобавить белые-прозранчые границы вокруг изображения - создаем новое изображение размером
    #180*180 пикселей и вставляем туда исходное изображение
    #изменим размер изображения
    im2 = Image.new('RGBA', (180, 180), color=(255, 255, 255, 0)) #создаем новое изображение формата RGBA, размером 180*180 пикселей, фон - белый, абсолютно прозрачный
    max_size = 180
    size = (min(max_size, max(im.size)),) * 2 #получаем кортеж размеров
    if size[0] < im.size[0] or size[1] < im.size[1]: #если ширина изображения im больше чем новое изображение im2
        im.thumbnail(size) #меняем размер изображения im до размеров изображения im2
    
    # Находим координаты верхнего левого угла, чтобы отцентрировать изображение im в изображении im2
    assert im2.size[0] >= im.size[0]
    assert im2.size[1] >= im.size[1]
    x = (im2.size[0] - im.size[0]) // 2
    y = (im2.size[1] - im.size[1]) // 2

    #Вставляем изображение
    im2.paste(im, (x, y))
    os.chdir(direct_save) #переходим в папку для сохранения итоговых изображений
    point = name[name.find('.') : ] 
    if point == 5:
        im2.save(name[ : -5] + '_edit.png', "PNG")
    else:
        im2.save(name[ : -4] + '_edit.png', "PNG")    
    return no_kompas #функция возвращает триггер наличия компаса в изображении

def two_point_one():
    os.chdir(way)
    a = os.listdir()
    os.chdir(save_w)
    for b in a:
        if b != 'kompas_white.png':
            try:
                os.mkdir(b + '_2.1')
            except:
                pass
    
    planirovki_list = []
    get_files_two_point_one(planirovki_list, way)
    
    for i in range (0, len(planirovki_list), 2): #идем по списку planirovki_list (перебор папок)
        for j in range (0, len(planirovki_list[i + 1])): #идем по списку planirovki_list (перебор файлов)
            if planirovki_list[i + 1][j].count('kompas') != True: #если название файла не содержит 'kompas' - файл нужно обрабатывать иначе это изображение-шаблон
                name = planirovki_list[i + 1][j] #name -строка состоящая из названия файла
            direct = way + planirovki_list[i] + '/' #путь к папке с исходными изображениями
            direct_save = save_w + planirovki_list[i] + '_2.1' + '/' #путь к папке для сохранения обработанных изображений
            find_kompas(name, direct, direct_save) #выполняем функцию find_kompas


#Пункт 2.2
planirovki_list = []
arr, arr_c = (0, 0, 0), (0, 0, 0)
img_list = []
k, no_kompas = 1, 0

def get_files_two_point_two(planirovki_list, way): #Функция будет возвращать список всех папок в каталоге и файлов в этих папках
    os.chdir(way)
    fol = os.listdir()
    folders = []
    for f in fol:
        if os.path.isdir(f) == True:
            folders.append(f)
    for i in range (0, len(folders)):
        if str(folders[i]).count('_') != True:
            os.chdir(way + str(folders[i]) + '/')
            planirovki_list.append(folders[i])
            planirovki_list.append(os.listdir()) 
    return planirovki_list

def with_kompas(direct, direct_save, name):
    os.chdir(direct)
    im = Image.open(name)
    im = im.convert("RGBA") #конвертируем изображение в RGB с альфа-каналом 
    width = im.size[0] #Определяем ширину. 
    height = im.size[1] #Определяем высоту. 
    pix = im.load() #Выгружаем значения пикселей.

    #делаем границы прозрачными
    opacity_level = 0 # Задаем уровень прозрачности границ вводится значения от 0 до 255
    center = (0, 2)
    yellow = (255, 255, 0, 0)
    ghost = im.getpixel(center)
    if ghost[3] > 100:
        ImageDraw.floodfill(im, center, yellow, thresh = 10) #заливаем все прозрачным фоном начина с 2 пикселя !Внимание на thresh!
    #обрезаем изображение
    left, up = 0, 0
    min_j, max_j = 0, height
    right, down = 0, 0
    min_i = 0 
    for i in range (width):
        for j in range (height):
            #print (pix[i, j][3])
            if pix[i, j][3] > 100 and left == 0: #ищем левую границу
                left = i 
                max_j =  j
            elif pix[i, j][3] > 100 and left != 0: 
                if j < max_j: #ищем верхнюю границу
                    up = j
                    max_j = j
                elif j > min_j: #ищем нижнюю границу
                    down =  j
                    min_j = j 
                elif i > min_i: #ищем правую границу
                    right = i
                    min_i = i
    area = (left, up, right, down)
    im = im.crop(area)
    #изменим размер изображения
    old_img = im
    new_img = Image.new('RGBA', (380, 455), color=(255, 255, 255, 0))
    old_img.thumbnail((380, 455))
    x = (new_img.size[0] - old_img.size[0]) // 2
    y = (new_img.size[1] - old_img.size[1]) // 2
    
    # paste image
    new_img.paste(old_img, (x, y))
    im2 = new_img
    os.chdir(direct_save)
    point = name[name.find('.') : ] 
    if point == 5:
        im2.save(name[ : -5] + '_edit.png', "PNG")
    else:
        im2.save(name[ : -4] + '_edit.png', "PNG")
    
def two_point_two():
    os.chdir(way)
    a = os.listdir()
    os.chdir(save_w)
    for b in a:
        if b.count('_') != True and b != 'kompas_white.png':
            try:
                os.mkdir(b + '_')
            except:
                pass
    get_files_two_point_two(planirovki_list, way)
    os.chdir(way)

    for i in range (0, len(planirovki_list), 2): #идем по списку planirovki_list (перебор папок)
        for j in range (0, len(planirovki_list[i + 1])): #идем по списку planirovki_list (перебор файлов)
            if planirovki_list[i + 1][j].count('kompas') != True: #если название файла не содержит 'kompas' - файл нужно обрабатывать иначе это изображение-шаблон
                name = planirovki_list[i + 1][j] #name -строка состоящая из названия файла
            direct = way + planirovki_list[i] + '/' #путь к папке с исходными изображениями
            direct_save = save_w + planirovki_list[i] + '_' + '/' #путь к папке для сохранения обработанных изображений
            with_kompas(direct, direct_save, name)

#Запускаем выполнение обработки типовок только если типовки есть в папке tipovki_to_update

os.chdir('/content/drive/My Drive/Colab Notebooks/examples/tipovki_to_update/')
new_tipovki = os.listdir()
if new_tipovki != []:
    tipovki() #запуск обработки типовок

#Запускаем выполнение обработки планировок только если планировки есть в папке planirovki_to_update
os.chdir('/content/drive/My Drive/Colab Notebooks/examples/planirovki_to_update/')
planirovki = os.listdir()
if planirovki != [] and planirovki != ['kompas_white.png']:
    two_point_one()
    two_point_two()
if new_tipovki == [] and planirovki == ['kompas_white.png']:
    print("Нет изображений для обработки")